In [2]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [1]:
# print(os.getenv("OPENAI_API_KEY"))
print ("API")

API


This script demonstrates how to use the OpenAI Python SDK to send a system instruction and a user prompt to an OpenAI model and print the generated response.
It initializes the OpenAI client using an API key from environment variables,calls the Responses API, and outputs the model’s text reply.

In [5]:
from openai import OpenAI

openai_client = OpenAI()

system_prompt = "You can make funny and original jokes."
user_prompt = "Tell me a joke about Alexey."

chat_messages = [
    {"role": "developer", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

response = openai_client.responses.create(
    model='gpt-4o-mini',
    input=chat_messages,
)

print(response.output_text)

Why did Alexey bring a ladder to the bar?

Because he heard the drinks were on the house!


With function caling:

This example defines a local Python function that generates a personalized joke and exposes it as a tool that an OpenAI model can call. The model receives system instructions and a user prompt, decides when to use the function, and returns a structured response that enables function-based agent behavior.


In [6]:
import random

def make_joke(name):
    jokes = [
        f"Why did {name} bring a pencil to the party? Because he wanted to draw some attention!",
        f"Did you hear about {name}'s bakery? Business is on a roll!",
        f"{name} walked into a library and asked for a burger. The librarian said, 'This is a library.' So {name} whispered, 'Can I get a burger?'",
        f"When {name} does push-ups, the Earth moves down.",
        f"{name} told a chemistry joke... but there was no reaction.",
    ]
    return random.choice(jokes)

print(make_joke("Alexey"))

make_joke_description = {
    "type": "function",
    "name": "make_joke",
    "description": "Generates a random personalized joke using the provided name.",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {
                "type": "string",
                "description": "The name to insert into the joke, personalizing the output.",
            }
        },
        "required": ["name"],
        "additionalProperties": False,
    },
}

### 

system_prompt = "You can make funny and original jokes. Find out the user's name to make the joke personalized."

user_prompt = "Tell me a joke about Alexey."

chat_messages = [
    {"role": "developer", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

response = openai_client.responses.create(
    model='gpt-4o-mini',
    input=chat_messages,
    tools=[make_joke_description]
)

response.output

When Alexey does push-ups, the Earth moves down.


[ResponseFunctionToolCall(arguments='{"name":"Alexey"}', call_id='call_enPZkLkPhpAYNS6w2s1lXZ2y', name='make_joke', type='function_call', id='fc_091313829b7f8721006942f70b677c8194b04f288703cdb8b4', status='completed')]

For orchestrating, we can use toyaikit.

In [7]:
from toyaikit.tools import Tools
from toyaikit.chat import IPythonChatInterface
from toyaikit.llm import OpenAIClient
from toyaikit.chat.runners import OpenAIResponsesRunner

tools_obj = Tools()
tools_obj.add_tool(make_joke, make_joke_description)

chat_interface = IPythonChatInterface()
openai_client = OpenAIClient(client=OpenAI())

runner = OpenAIResponsesRunner(
    tools=tools_obj,
    developer_prompt=system_prompt,
    chat_interface=chat_interface,
    llm_client=openai_client
)

runner.run()

You: tell me a joke


-> Response received


You: Olivera


-> Response received


-> Response received


You: Yes


-> Response received


You: stop


Chat ended.


LoopResult(new_messages=[EasyInputMessage(content="You can make funny and original jokes. Find out the user's name to make the joke personalized.", role='developer', type=None), EasyInputMessage(content='tell me a joke', role='user', type=None), ResponseOutputMessage(id='msg_0f01ec9356dfd031006942f8e753788195b192a866bd743de7', content=[ResponseOutputText(annotations=[], text="What's your name? I can personalize a joke just for you!", type='output_text', logprobs=[])], role='assistant', status='completed', type='message'), EasyInputMessage(content='Olivera', role='user', type=None), ResponseFunctionToolCall(arguments='{"name":"Olivera"}', call_id='call_xEh88y781yhUkOztGFhmYQg5', name='make_joke', type='function_call', id='fc_0f01ec9356dfd031006942f8f8ebec81958f156c8cd699bb37', status='completed'), {'type': 'function_call_output', 'call_id': 'call_xEh88y781yhUkOztGFhmYQg5', 'output': '"Why did Olivera bring a pencil to the party? Because he wanted to draw some attention!"'}, ResponseOutp